In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import nuSQUIDSpy as nsq

In [9]:
units = nsq.Const()
gr = nsq.GlashowResonanceCrossSection()
dis = nsq.NeutrinoDISCrossSectionsFromTables()
tds = nsq.TauDecaySpectra()

In [68]:
def TotalNeutrinoCrossSection(enu, 
                              flavor = nsq.NeutrinoCrossSections_NeutrinoFlavor.electron,
                              neutype = nsq.NeutrinoCrossSections_NeutrinoType.neutrino):
    return np.sum([dis.TotalCrossSection(enu/units.GeV,flavor,neutype,interaction) for interaction in
            [nsq.NeutrinoCrossSections_Current.CC,nsq.NeutrinoCrossSections_Current.NC]])

def DifferentialOutGoingLeptonDistribution(enu_in,enu_out,
                                       flavor = nsq.NeutrinoCrossSections_NeutrinoFlavor.tau,
                                       neutype = nsq.NeutrinoCrossSections_NeutrinoType.neutrino,
                                       interaction = nsq.NeutrinoCrossSections_Current.NC
                                    ):
    total = dis.TotalCrossSection(enu_in/units.GeV,flavor,neutype,interaction)
    diff = dis.SingleDifferentialCrossSection(enu_in/units.GeV,enu_out/units.GeV,flavor,neutype,interaction)
    return diff/total

def DistributionOfTauDecays(tau_initial,nutau_initial):
    # tau dies and resurrects as a neutrino #
    # this returns dN/dz with z = nutau_initial/tau_initial and is such that \int_0^1 dNdz = 1
    return tds.TauDecayToAll(tau_initial,nutau_initial)

In [124]:
Etau = 100.
zz = np.linspace(0.0,1.0,500)
dNTaudz = lambda z: tds.TauDecayToAll(Etau, Etau*z)
TauDecayWeights = np.array(map(dNTaudz,zz))
TauDecayWeights = weights/np.sum(TauDecayWeights)

yy = np.linspace(0.0,1.0,100)

proton_mass = 1.*units.GeV

In [150]:
class CasinoEvent(object):
    def __init__(self, particle_id, energy, position):
        self.particle_id = particle_id
        self.energy = energy
        self.position = position
        self.SetParticleProperties()
        
    def SetParticleProperties(self):
        if self.particle_id == "tau_neutrino":
            self.mass = 0.0
            self.lifetime = np.inf
        if self.particle_id == "tau":
            self.mass = 1.7*units.GeV
            self.lifetime = 1.*units.sec
            
    def GetParticleId(self):
        return self.particle_id
    
    def GetLifetime(self):
        return self.lifetime
    
    def GetMass(self):
        return self.mass
    
    def GetBoostFactor(self):
        if self.mass > 0.:
            return self.energy/self.mass
        else:
            return np.inf
            
    def GetDecayProbability(self,dL):
        boost_factor = self.GetBoostFactor()
        return dL/(boost_factor*self.lifetime)
    
    def GetInteractionLength(self,density):
        # this should be actually divided by average of proton and neutron mass
        return TotalNeutrinoCrossSection(self.energy)*density/(proton_mass)
    
    def GetInteractionProbability(self,dL,density):
        return dL/self.GetInteractionLength(density)
    
    def DecayParticle(self):
        if self.particle_id == "tau_neutrino":
            return
        if self.particle_id == "tau":
            self.energy = self.energy*np.random.choice(zz, p=TauDecayWeights)
            self.particle_id = "tau_neutrino"
            self.SetParticleProperties()
            return
            
    def InteractParticle(self):
        if self.particle_id == "tau_neutrino":
            dNdEle = lambda y: DifferentialOutGoingLeptonDistribution(self.energy,self.energy*y)
            NeutrinoInteractionWeights = map(dNdEle,yy)
            NeutrinoInteractionWeights = NeutrinoInteractionWeights/np.sum(NeutrinoInteractionWeights)
            self.energy = self.energy*np.random.choice(yy, p=NeutrinoInteractionWeights)
            self.particle_id = "tau"
            self.SetParticleProperties()
            return
        if self.particle_id == "tau":
            # proposal comes here
            return
        
    def PrintParticleProperties(self):
        print self.particle_id, self.energy, self.position

In [151]:
TotalDistance = 100*units.km
ProposedDistanceStep = 1.*units.km 
density = 1.*units.gr/units.cm**3

In [152]:
FirstEvent = CasinoEvent("tau_neutrino",1e7*units.GeV,0.0)
EventCollection = [FirstEvent]

while(not np.any(map(lambda e: e.position > TotalDistance, EventCollection))):
    for event in EventCollection:
        p = np.random.random_sample()
        if event.GetDecayProbability(ProposedDistanceStep) > p:
            # do decay
            print "Decayed"
            event.DecayParticle()
        p = np.random.random_sample()
        if event.GetInteractionProbability(ProposedDistanceStep,density) > p:
            print "Interacted"
            event.InteractParticle()
    # move everything forward
    for event in EventCollection:
        event.position += ProposedDistanceStep

Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted
Interacted

In [153]:
event.PrintParticleProperties()

tau 1.31313131313e+15 5.11840824678e+11


In [155]:
1.31313131313e+15/(1e7*units.GeV)

0.131313131313